In [34]:
import spacy
import pandas as pd
from spacy.tokens import DocBin
from spacy.util import filter_spans

In [35]:
train_data = pd.read_json('../data/train.jsonl', lines=True)
test_data = pd.read_json('../data/test.jsonl', lines=True)

In [36]:
train_data.head()

,ners,sentences,id
0,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS...",Бостон взорвали Тамерлан и Джохар Царнаевы из ...,0
1,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],...",Умер избитый до комы гитарист и сооснователь г...,1
2,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O...",Путин подписал распоряжение о выходе России из...,2
3,"[[0, 11, PERSON], [36, 47, PROFESSION], [49, 6...",Бенедикт XVI носил кардиостимулятор\nПапа Римс...,3
4,"[[0, 4, PERSON], [17, 29, ORGANIZATION], [48, ...",Обама назначит в Верховный суд латиноамериканк...,4


In [37]:
test_data.head()

,senences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588


In [38]:
test_data.rename(columns={"senences": "sentences"}, inplace=True)

In [39]:
nlp = spacy.blank("ru")
doc_bin = DocBin()

In [40]:
for ind, row in train_data.iterrows():
    tokens = row['sentences']
    doc = nlp.make_doc(tokens)
    ents = []
    for ner in row['ners']:
        start, end, label = ner

        span = doc.char_span(start, end+1, label=label, alignment_mode="contract")
        if span is None or span.text != span.text.strip():
            continue

        ents.append(span)
    doc.ents = filter_spans(ents)
    doc_bin.add(doc)

doc_bin.to_disk("./train_data.spacy")

In [41]:
!python -m spacy train config.cfg --output ./ --paths.train ./train_data.spacy --paths.dev ./train_data.spacy

]4;0;#090d0e\]4;1;#B55134\]4;2;#937157\]4;3;#AB886A\]4;4;#CC9A6F\]4;5;#386093\]4;6;#6690B1\]4;7;#dfcfc5\]4;8;#9c9089\]4;9;#B55134\]4;10;#937157\]4;11;#AB886A\]4;12;#CC9A6F\]4;13;#386093\]4;14;#6690B1\]4;15;#dfcfc5\]10;#dfcfc5\]11;#090d0e\]12;#dfcfc5\]13;#dfcfc5\]17;#dfcfc5\]19;#090d0e\]4;232;#090d0e\]4;256;#dfcfc5\]708;#090d0e\ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.76    0.53    0.31    1.72    0.01
  0     200       1558.86  15512.17   32.42   42.83   26.08    0.32
  0     400        755.71  11423.99   47.16   58.01   39.73    0.

In [42]:
nlp_ner = spacy.load("model-best")

ners_list = []
for index, row in test_data.iterrows():
    doc = nlp_ner(row['sentences'])
    ners_list.append([(ent.start_char, ent.end_char - 1, ent.label_) for ent in doc.ents])

test_data['ners'] = ners_list

In [43]:
# test_data.drop(columns=['sentences'], inplace=True)

with open('test.jsonl', "w") as f:
    f.write(test_data.to_json(orient='records', lines=True, force_ascii=False))